**Hatespeech Classifier**

Data from https://github.com/aitor-garcia-p/hate-speech-dataset

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.text import *

Obviously, modify the path below to reflect where your data is stored

In [ ]:
path = pathlib.PosixPath('/home/andy/Documents/nlp/data/hate-speech-dataset')

In [ ]:
bs=48
hate_data_lm = (TextList.from_folder(path)
           #Inputs: all the text files in path
            .filter_by_folder(include=['unsup','train','test']) 
           #We may have other temp folders that contain text files so we only keep what's in train and test
            .split_by_rand_pct(0.1)
           #We randomly split and keep 10% (10,000 reviews) for validation
            .label_for_lm()           
           #We want to do a language model so we label accordingly
            .databunch(bs=bs))
hate_data_lm.save('hate_data_lm.pkl')
hate_data_lm = load_data(path, 'hate_data_lm.pkl', bs=bs)

In [ ]:
hate_data_lm.show_batch()

In [ ]:
learn = language_model_learner(hate_data_lm, AWD_LSTM, drop_mult=0.3)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot(skip_end=15, suggestion=True)

In [ ]:
learn.fit_one_cycle(1, 7e-2, moms=(0.8,0.7))

In [ ]:
learn.save('hate_fit_head')

In [ ]:
learn.load('hate_fit_head')

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(10, 1e-3, moms=(0.8,0.7)) # gets about 0.28

In [ ]:
learn.save('hate-fine-tuned')

In [ ]:
learn.save_encoder('hate-fine-tuned_enc')

In [ ]:
df = pd.read_csv('/home/andy/Documents/nlp/data/hate-speech-dataset/annotations_metadata.csv')
df.head()
df2 = df.set_index('file_id')
df2.head()
df2.loc['12834217_1','label']

In [ ]:
hate_data_clas = (TextList.from_folder(path, vocab=hate_data_lm.vocab)
             #grab all the text files in path
             .split_by_folder(valid='train')
             #split by train and valid folder (that only keeps 'train' and 'test' so no need to filter)
             #.label_from_csv('/home/andy/Documents/nlp/data/hate-speech-dataset/annotations_metadata.csv')
             #.label_from_func(lambda x : df2['label'].iloc[ (os.path.basename(x))[:-4] ])
             .label_from_func(lambda x : df2.loc[(os.path.basename(x))[:-4], 'label'])
             .databunch(bs=bs))

hate_data_clas.save('hate_data_clas.pkl')

In [ ]:
hate_data_clas.show_batch()

In [ ]:
learn = text_classifier_learner(hate_data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('hate-fine-tuned_enc')

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot(suggestion=True)

In [ ]:
learn.fit_one_cycle(1, 3.63e-3, moms=(0.8,0.7)) # from suggestion, gets about 0.76

In [ ]:
learn.save('hate-first')

In [ ]:
learn.load('hate-first')

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7)) # gets about 0.80 accuracy

In [ ]:
learn.save('hate-unfrozen-to-3')

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot(suggestion=True)

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, 2e-5, moms=(0.8,0.7))